In [25]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
green = (0, 255, 0)
red = (0, 0, 255)

In [26]:
def show_image(img):
    cv2.imshow('image', img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows() 

In [55]:
img = cv2.imread('Images/gauge-2.jpg')
height, width = img.shape[0:2]
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [23]:



circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20, np.array([]), 100, 50, int(height*0.35), int(height*0.48))

print(circles)

x, y, r = circles[0][0]


detected = cv2.circle(img, (int(x), int(y)), int(r), red, 3, cv2.LINE_AA)  # draw circle


show_image(detected)

# plt.imshow(img)
# gray.shape

[[[127.5 134.5  95.5]]]


In [56]:
threshold, result = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)


dst = cv2.Canny(gray, 50, 200, None, 3)

lines = cv2.HoughLinesP(dst, 1, np.pi / 180, 100, None, 10, 0)

if lines is not None:
    for i in range(0, len(lines)):
        l = lines[i][0]
        cv2.line(img, (l[0], l[1]), (l[2], l[3]), (0,255,0), 1, cv2.LINE_AA)

show_image(img)